# Pre-process POP909 for Polyphemus
- split 909 into phrases instead of bars
- Phrases are of varying bar-length --> Add padding??
- Use 4/4 only

In [1]:
import re
import os
import time
import sys
import multiprocessing
import itertools
import argparse
from itertools import product

import numpy as np
from tqdm import tqdm
import pypianoroll as pproll
import muspy

In [22]:
midi_folder = "POP909"
structure_folder = "POP909_structure"

In [43]:
import constants
from constants import PitchToken, DurationToken


def split_string(s):
    # This regex pattern matches a letter followed by one or more digits
    pattern = re.compile(r'[a-zA-Z]\d+')
    # Find all matches in the string
    matches = pattern.findall(s)
    return matches


MAX_PHRASE_LEN = 8


def split_song_into_phrases(pproll_song, phrases):
    muspy_conv = muspy.from_pypianoroll(pproll_song)

    n_beats = [track.notes[-1].time//8 for track in muspy_conv.tracks]
    num_beats_per_phrase = 4*MAX_PHRASE_LEN
    num_bars = max(n_beats)//4
    
    print(f"Num bars: {num_bars}")

    tracks = []
    # For each piece, interate through each track
    for track in muspy_conv.tracks:
        # For each track, split track into bars
        bars = [[] for i in range(num_bars+1)]

        for note in track.notes:
            bar_num = note.time//(4*8)
            bars[bar_num].append(note)
        tracks.append(bars)
        
    # Split one song into phrases
    start_bar_idx = 0
    phrase_songs=[]
    for phrase in phrases: 
        num_bars = int(phrase[1:])
        end_bar_idx = start_bar_idx + min(num_bars, MAX_PHRASE_LEN) # Cut end bar index to maximum phrase len if it exceeds that.
        phrase_song = muspy.Music(resolution=8)
        phrase_tracks = [muspy.Track(notes=[note for t in track[start_bar_idx:end_bar_idx] for note in t]) for track in tracks]
        phrase_song.tracks = phrase_tracks
        for idx, track_type in enumerate(["MELODY", "BRIDGE", "PIANO"]):
            phrase_song.tracks[idx].name = track_type
        phrase_songs.append(phrase_song)
        start_bar_idx += num_bars
        
    return phrase_songs


def process_track_notes(tracks_notes, ):
    tracks_content = []
    tracks_structure = []

    max_phrase_len_res = 4*resolution*MAX_PHRASE_LEN # tokens in max number of bars

    for notes in tracks_notes:

        # track_content: length x MAX_SIMU_TOKENS x 2
        # This is used as a basis to build the final content tensors for
        # each sequence.
        # The last dimension contains pitches and durations. int16 is enough
        # to encode small to medium duration values.
        track_content = np.zeros((max_phrase_len_res, constants.MAX_SIMU_TOKENS, 2), 
                                np.int16)

        track_content[:, :, 0] = PitchToken.PAD.value
        track_content[:, 0, 0] = PitchToken.SOS.value
        track_content[:, :, 1] = DurationToken.PAD.value
        track_content[:, 0, 1] = DurationToken.SOS.value

        # Keeps track of how many notes have been stored in each timestep
        # (int8 imposes MAX_SIMU_TOKENS < 256)
        notes_counter = np.ones(max_phrase_len_res, dtype=np.int8)

        # Todo: np.put_along_axis?
        for note in notes:
            # Insert note in the lowest position available in the timestep

            t = note.time%max_phrase_len_res
            
            # if t >= max_phrase_len_res:
            #     # Skip note if it exceeds max phrase length
            #     continue

            if notes_counter[t] >= constants.MAX_SIMU_TOKENS-1:
                # Skip note if there is no more space
                continue

            pitch = max(min(note.pitch, constants.MAX_PITCH_TOKEN), 0)
            track_content[t, notes_counter[t], 0] = pitch
            dur = max(min(note.duration, constants.MAX_DUR_TOKEN + 1), 1)
            track_content[t, notes_counter[t], 1] = dur-1
            notes_counter[t] += 1
        # print(f"num notes: {notes_counter}")
        # Add EOS token
        t_range = np.arange(0, max_phrase_len_res)
        track_content[t_range, notes_counter, 0] = PitchToken.EOS.value
        track_content[t_range, notes_counter, 1] = DurationToken.EOS.value

        # Get track activations, a boolean tensor indicating whether notes
        # are being played in a timestep (sustain does not count)
        # (needed for graph rep.)
        activations = np.array(notes_counter-1, dtype=bool)

        tracks_content.append(track_content)
        tracks_structure.append(activations)
        
        # n_tracks x length x MAX_SIMU_TOKENS x 2
        c_tensor = np.stack(tracks_content, axis=0)

        # n_tracks x length
        s_tensor = np.stack(tracks_structure, axis=0)
    
    return c_tensor, s_tensor


In [32]:
label_paths = ["human_label1"]

resolution = 8

for song_idx in tqdm(os.listdir(midi_folder)):
    saved_samples = 0 # Restart count for every song
    for label_path in label_paths:
        try:
            midi_filepath = f"{midi_folder}/{song_idx}/{song_idx}.mid"
            
            # print(midi_filepath)
            
            pproll_song = pproll.read(midi_filepath, resolution=resolution)
            muspy_song = muspy.read(midi_filepath)
            
            # Only accept songs that have a time signature of 4/4 and no time changes
            for t in muspy_song.time_signatures:
                # print(t)
                if t.numerator == 3 or t.denominator != 4:
                    print(f"Song skipped {midi_filepath} ({t.numerator}/{t.denominator} time signature)")
                    continue
                else:
                    print(f"Song accepted! {midi_filepath} with time signature {t.numerator}/{t.denominator}")


            f = open(f"{structure_folder}/{song_idx}/{label_path}.txt", "r")
            structure = f.read()
            phrases = split_string(structure)
            # print(f"Structure: {structure}")
            # print(f"Phrase length: {len(phrases)}")
                        
            phrase_songs = split_song_into_phrases(pproll_song, phrases)
            # print(f"Phrase songs length: {len(phrase_songs)}")
            
            for phrase_song in phrase_songs:
            
                tracks_notes = [track.notes for track in phrase_song.tracks]
                c_tensor, s_tensor = process_track_notes(tracks_notes)
                # print(f"C tensor shape: {c_tensor.shape}, S tensor shape: {s_tensor.shape}")
            
                # Save sample (content and structure) to file
                dest_dir = "preprocessed_909"
                filename = os.path.basename(midi_filepath)

                sample_filepath = os.path.join(
                    dest_dir, filename+str(saved_samples))
                np.savez(sample_filepath, c_tensor=c_tensor, s_tensor=s_tensor)
                print(f"saved to {sample_filepath}")
            
                saved_samples += 1
            
        except:
            print(f"Skipped {song_idx}")
            continue

  0%|          | 1/910 [00:00<04:13,  3.59it/s]

Song accepted! POP909/295/295.mid with time signature 1/4
Num bars: 85
saved to preprocessed_909/295.mid0
saved to preprocessed_909/295.mid1
saved to preprocessed_909/295.mid2
saved to preprocessed_909/295.mid3
saved to preprocessed_909/295.mid4
saved to preprocessed_909/295.mid5
saved to preprocessed_909/295.mid6
saved to preprocessed_909/295.mid7
saved to preprocessed_909/295.mid8
saved to preprocessed_909/295.mid9
saved to preprocessed_909/295.mid10
saved to preprocessed_909/295.mid11
saved to preprocessed_909/295.mid12
saved to preprocessed_909/295.mid13
saved to preprocessed_909/295.mid14
saved to preprocessed_909/295.mid15
saved to preprocessed_909/295.mid16
Song accepted! POP909/183/183.mid with time signature 4/4
Num bars: 67
saved to preprocessed_909/183.mid0


  0%|          | 2/910 [00:00<03:15,  4.65it/s]

saved to preprocessed_909/183.mid1
saved to preprocessed_909/183.mid2
saved to preprocessed_909/183.mid3
saved to preprocessed_909/183.mid4
saved to preprocessed_909/183.mid5
saved to preprocessed_909/183.mid6
saved to preprocessed_909/183.mid7
saved to preprocessed_909/183.mid8
saved to preprocessed_909/183.mid9
saved to preprocessed_909/183.mid10
saved to preprocessed_909/183.mid11
saved to preprocessed_909/183.mid12
saved to preprocessed_909/183.mid13
saved to preprocessed_909/183.mid14
saved to preprocessed_909/183.mid15
saved to preprocessed_909/183.mid16
Song accepted! POP909/567/567.mid with time signature 1/4


  0%|          | 4/910 [00:00<02:42,  5.57it/s]

...

Song accepted! POP909/073/073.mid with time signature 1/4
Num bars: 85
saved to preprocessed_909/073.mid0
saved to preprocessed_909/073.mid1
saved to preprocessed_909/073.mid2
saved to preprocessed_909/073.mid3
saved to preprocessed_909/073.mid4
saved to preprocessed_909/073.mid5
saved to preprocessed_909/073.mid6
saved to preprocessed_909/073.mid7
saved to preprocessed_909/073.mid8
saved to preprocessed_909/073.mid9
saved to preprocessed_909/073.mid10
Song accepted! POP909/829/829.mid with time signature 1/4
Num bars: 107
saved to preprocessed_909/829.mid0
saved to preprocessed_909/829.mid1
saved to preprocessed_909/829.mid2
saved to preprocessed_909/829.mid3
saved to preprocessed_909/829.mid4
saved to preprocessed_909/829.mid5
saved to preprocessed_909/829.mid6
saved to preprocessed_909/829.mid7
saved to preprocessed_909/829.mid8
saved to preprocessed_909/829.mid9
saved to preprocessed_909/829.mid10
saved to preprocessed_909/829.mid11
saved to preprocessed_909/829.mid12
Song accepted

100%|█████████▉| 909/910 [02:46<00:00,  9.13it/s]

Num bars: 65
saved to preprocessed_909/174.mid0
saved to preprocessed_909/174.mid1
saved to preprocessed_909/174.mid2
saved to preprocessed_909/174.mid3
saved to preprocessed_909/174.mid4
saved to preprocessed_909/174.mid5
saved to preprocessed_909/174.mid6
saved to preprocessed_909/174.mid7
saved to preprocessed_909/174.mid8
saved to preprocessed_909/174.mid9
Song accepted! POP909/061/061.mid with time signature 1/4
Num bars: 91
saved to preprocessed_909/061.mid0
saved to preprocessed_909/061.mid1
saved to preprocessed_909/061.mid2
saved to preprocessed_909/061.mid3
saved to preprocessed_909/061.mid4
saved to preprocessed_909/061.mid5
saved to preprocessed_909/061.mid6
saved to preprocessed_909/061.mid7
saved to preprocessed_909/061.mid8
saved to preprocessed_909/061.mid9
saved to preprocessed_909/061.mid10
saved to preprocessed_909/061.mid11
saved to preprocessed_909/061.mid12
saved to preprocessed_909/061.mid13
saved to preprocessed_909/061.mid14
Song accepted! POP909/160/160.mid wi

100%|██████████| 910/910 [02:46<00:00,  5.48it/s]

Song accepted! POP909/900/900.mid with time signature 1/4
Num bars: 71
saved to preprocessed_909/900.mid0
saved to preprocessed_909/900.mid1
saved to preprocessed_909/900.mid2
saved to preprocessed_909/900.mid3
saved to preprocessed_909/900.mid4
saved to preprocessed_909/900.mid5
saved to preprocessed_909/900.mid6
saved to preprocessed_909/900.mid7
saved to preprocessed_909/900.mid8
saved to preprocessed_909/900.mid9
saved to preprocessed_909/900.mid10
saved to preprocessed_909/900.mid11
saved to preprocessed_909/900.mid12
saved to preprocessed_909/900.mid13


In [39]:
song_idxs = [1, 2, 3, 4, 5]
midi_dataset_dir = "test"
structure_dir = "st"
preprocessed_dir = "pd"
n_bars = 2
resolution = 8
fn = itertools.chain(
        (os.path.join(midi_dataset_dir, f"{song_idx}/{song_idx}.mid"), structure_dir, preprocessed_dir, n_bars, resolution)
            for song_idx in song_idxs
    )
list(fn)

[('test/1/1.mid', 'st', 'pd', 2, 8),
 ('test/2/2.mid', 'st', 'pd', 2, 8),
 ('test/3/3.mid', 'st', 'pd', 2, 8),
 ('test/4/4.mid', 'st', 'pd', 2, 8),
 ('test/5/5.mid', 'st', 'pd', 2, 8)]

In [36]:
list(fn)

NameError: name 'structure_dir' is not defined

## Debug

In [3]:
filepath = "POP909/001/001.mid"
structure_path = "POP909_structure/001/human_label1.txt"

In [25]:
pproll_song = pproll.read(filepath, resolution=8)
muspy_song = muspy.read(filepath)

In [26]:
f = open(f"{structure_path}", "r")
structure = f.read()
phrases = split_string(structure)

In [27]:
phrases

['i4',
 'A4',
 'B8',
 'A4',
 'A4',
 'b4',
 'B8',
 'A4',
 'A4',
 'b4',
 'b4',
 'A4',
 'A4',
 'b4',
 'A4',
 'o3']

In [316]:
MAX_PHRASE_LEN = 8


def split_song_into_phrases(pproll_song, phrases):
    muspy_conv = muspy.from_pypianoroll(pproll_song)

    n_beats = [track.notes[-1].time//8 for track in muspy_conv.tracks]
    num_beats_per_phrase = 4*MAX_PHRASE_LEN
    num_bars = max(n_beats)//4

    tracks = []
    # For each piece, interate through each track
    for track in muspy_conv.tracks:
        # For each track, split track into bars
        bars = [[] for i in range(num_bars+1)]

        for note in track.notes:
            bar_num = note.time//(4*8)
            bars[bar_num].append(note)
        tracks.append(bars)
        
    # Split one song into phrases
    start_bar_idx = 0
    phrase_songs=[]
    for phrase in phrases: 
        num_bars = int(phrase[1:])
        end_bar_idx = start_bar_idx+num_bars
        phrase_song = muspy.Music(resolution=8)
        phrase_tracks = [muspy.Track(notes=[note for t in track[start_bar_idx:end_bar_idx] for note in t]) for track in tracks]
        phrase_song.tracks = phrase_tracks
        for idx, track_type in enumerate(["MELODY", "BRIDGE", "PIANO"]):
            phrase_song.tracks[idx].name = track_type
        phrase_songs.append(phrase_song)
        start_bar_idx = end_bar_idx
        
    return phrase_songs

In [29]:
len(phrases)

16

In [44]:
phrase_songs = split_song_into_phrases(pproll_song, phrases)
len(phrase_songs)

Num bars: 72


16

In [45]:
phrase_songs[0].tracks

[Track(program=0, is_drum=False, name='MELODY'),
 Track(program=0, is_drum=False, name='BRIDGE', notes=[Note(time=29, pitch=66, duration=3, velocity=64), Note(time=33, pitch=75, duration=3, velocity=64), Note(time=37, pitch=73, duration=1, velocity=64), ...]),
 Track(program=0, is_drum=False, name='PIANO', notes=[Note(time=33, pitch=47, duration=10, velocity=64), Note(time=35, pitch=54, duration=9, velocity=64), Note(time=37, pitch=59, duration=5, velocity=64), ...])]

In [46]:
phrase_songs[5].tracks

[Track(program=0, is_drum=False, name='MELODY'),
 Track(program=0, is_drum=False, name='BRIDGE', notes=[Note(time=793, pitch=73, duration=1, velocity=64), Note(time=799, pitch=80, duration=1, velocity=64), Note(time=801, pitch=82, duration=2, velocity=64), ...]),
 Track(program=0, is_drum=False, name='PIANO', notes=[Note(time=769, pitch=42, duration=28, velocity=64), Note(time=769, pitch=54, duration=5, velocity=64), Note(time=769, pitch=58, duration=4, velocity=64), ...])]

In [47]:
phrase_songs[10].tracks

[Track(program=0, is_drum=False, name='MELODY'),
 Track(program=0, is_drum=False, name='BRIDGE', notes=[Note(time=1552, pitch=73, duration=1, velocity=64), Note(time=1553, pitch=75, duration=3, velocity=64), Note(time=1560, pitch=68, duration=1, velocity=64), ...]),
 Track(program=0, is_drum=False, name='PIANO', notes=[Note(time=1537, pitch=42, duration=28, velocity=64), Note(time=1537, pitch=54, duration=5, velocity=64), Note(time=1537, pitch=58, duration=4, velocity=64), ...])]

In [57]:
tracks_notes = [track.notes for track in phrase_songs[5].tracks]
resolution=8

# Obtain length of subsong (maximum of each track's length)
length = 0
for notes in tracks_notes:
    track_length = max(note.end for note in notes) if notes else 0
    length = max(length, track_length)
length += 1


# Add timesteps until length is a multiple of resolution
length = length if length % (4*resolution) == 0 \
    else length + (4*resolution-(length % (4*resolution)))

In [51]:
import constants
from constants import PitchToken, DurationToken


In [67]:
def process_track_notes(tracks_notes):
    tracks_content = []
    tracks_structure = []

    max_phrase_len_res = 4*resolution*MAX_PHRASE_LEN

    for notes in tracks_notes:

        # track_content: length x MAX_SIMU_TOKENS x 2
        # This is used as a basis to build the final content tensors for
        # each sequence.
        # The last dimension contains pitches and durations. int16 is enough
        # to encode small to medium duration values.
        track_content = np.zeros((max_phrase_len_res, constants.MAX_SIMU_TOKENS, 2), 
                                np.int16)

        track_content[:, :, 0] = PitchToken.PAD.value
        track_content[:, 0, 0] = PitchToken.SOS.value
        track_content[:, :, 1] = DurationToken.PAD.value
        track_content[:, 0, 1] = DurationToken.SOS.value

        # Keeps track of how many notes have been stored in each timestep
        # (int8 imposes MAX_SIMU_TOKENS < 256)
        notes_counter = np.ones(max_phrase_len_res, dtype=np.int8)

        # Todo: np.put_along_axis?
        for note in notes:
            # Insert note in the lowest position available in the timestep

            t = note.time

            if t >= max_phrase_len_res:
                continue

            if notes_counter[t] >= constants.MAX_SIMU_TOKENS-1:
                # Skip note if there is no more space
                continue

            pitch = max(min(note.pitch, constants.MAX_PITCH_TOKEN), 0)
            track_content[t, notes_counter[t], 0] = pitch
            dur = max(min(note.duration, constants.MAX_DUR_TOKEN + 1), 1)
            track_content[t, notes_counter[t], 1] = dur-1
            notes_counter[t] += 1
        # print(f"num notes: {notes_counter}")
        # Add EOS token
        t_range = np.arange(0, max_phrase_len_res)
        track_content[t_range, notes_counter, 0] = PitchToken.EOS.value
        track_content[t_range, notes_counter, 1] = DurationToken.EOS.value

        # Get track activations, a boolean tensor indicating whether notes
        # are being played in a timestep (sustain does not count)
        # (needed for graph rep.)
        activations = np.array(notes_counter-1, dtype=bool)

        tracks_content.append(track_content)
        tracks_structure.append(activations)
        
        # n_tracks x length x MAX_SIMU_TOKENS x 2
        c_tensor = np.stack(tracks_content, axis=0)

        # n_tracks x length
        s_tensor = np.stack(tracks_structure, axis=0)
    
    return c_tensor, s_tensor


In [68]:
tracks_notes = [track.notes for track in phrase_songs[2].tracks]

c_tensor, s_tensor = process_track_notes(tracks_notes)

In [69]:
s_tensor

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
      

In [2]:
# Save sample (content and structure) to file
saved_samples=0
dest_dir = "preprocessed_909"
filename = os.path.basename(filepath)

sample_filepath = os.path.join(
    dest_dir, filename+str(saved_samples))
np.savez(sample_filepath, c_tensor=c_tensor, s_tensor=s_tensor)
print(f"saved to {sample_filepath}")

NameError: name 'filepath' is not defined

In [101]:
tracks[1]

[Music(metadata=Metadata(schema_version='0.1'), resolution=480, tracks=[Track(program=0, is_drum=False, notes=[Note(time=1720, pitch=66, duration=879, velocity=118), Note(time=1720, pitch=66, duration=204, velocity=121)])]),
 Music(metadata=Metadata(schema_version='0.1'), resolution=480, tracks=[Track(program=0, is_drum=False, notes=[Note(time=40, pitch=75, duration=237, velocity=121)])]),
 Music(metadata=Metadata(schema_version='0.1'), resolution=480, tracks=[Track(program=0, is_drum=False, notes=[Note(time=280, pitch=73, duration=131, velocity=118)])]),
 Music(metadata=Metadata(schema_version='0.1'), resolution=480, tracks=[Track(program=0, is_drum=False, notes=[Note(time=520, pitch=71, duration=148, velocity=123)])]),
 Music(metadata=Metadata(schema_version='0.1'), resolution=480, tracks=[Track(program=0, is_drum=False, notes=[Note(time=760, pitch=80, duration=95, velocity=118)])]),
 Music(metadata=Metadata(schema_version='0.1'), resolution=480, tracks=[Track(program=0, is_drum=Fals

In [87]:
muspy_song.tracks[1].notes

[Note(time=1720, pitch=66, duration=204, velocity=121),
 Note(time=1960, pitch=75, duration=237, velocity=121),
 Note(time=2200, pitch=73, duration=131, velocity=118),
 Note(time=2440, pitch=71, duration=148, velocity=123),
 Note(time=2680, pitch=80, duration=95, velocity=118),
 Note(time=2800, pitch=82, duration=115, velocity=104),
 Note(time=2920, pitch=80, duration=597, velocity=121),
 Note(time=3640, pitch=66, duration=164, velocity=118),
 Note(time=3880, pitch=75, duration=248, velocity=118),
 Note(time=4120, pitch=73, duration=374, velocity=118),
 Note(time=4600, pitch=70, duration=746, velocity=118),
 Note(time=5560, pitch=73, duration=142, velocity=112),
 Note(time=5800, pitch=78, duration=122, velocity=118),
 Note(time=5920, pitch=80, duration=59, velocity=118),
 Note(time=6040, pitch=82, duration=421, velocity=118),
 Note(time=6520, pitch=73, duration=99, velocity=78),
 Note(time=6760, pitch=78, duration=104, velocity=115),
 Note(time=6880, pitch=80, duration=61, velocity=118

In [56]:
tracks_notes = [track.notes for track in muspy_song.tracks]

# Obtain length of subsong (maximum of each track's length)
length = 0
for notes in tracks_notes:
    track_length = max(note.end for note in notes) if notes else 0
    length = max(length, track_length)
length += 1

In [57]:
length

134562

In [58]:
resolution=8
# Add timesteps until length is a multiple of resolution
length = length if length % (4*resolution) == 0 \
    else length + (4*resolution-(length % (4*resolution)))

In [59]:
length

134592

In [23]:
muspy_song.tracks[0].notes[0]

Note(time=9160, pitch=61, duration=69, velocity=115)

In [15]:
pproll_song

Multitrack(name=None, resolution=8, tempo=array(shape=(7080, 1), dtype=float64), downbeat=array(shape=(2360, 1), dtype=bool), tracks=[StandardTrack(name='MELODY', program=0, is_drum=False, pianoroll=array(shape=(2360, 128), dtype=uint8)), StandardTrack(name='BRIDGE', program=0, is_drum=False, pianoroll=array(shape=(2360, 128), dtype=uint8)), StandardTrack(name='PIANO', program=0, is_drum=False, pianoroll=array(shape=(2360, 128), dtype=uint8))])

In [16]:
pproll_song.tracks

[StandardTrack(name='MELODY', program=0, is_drum=False, pianoroll=array(shape=(3792, 128), dtype=uint8)),
 StandardTrack(name='BRIDGE', program=0, is_drum=False, pianoroll=array(shape=(3792, 128), dtype=uint8)),
 StandardTrack(name='PIANO', program=0, is_drum=False, pianoroll=array(shape=(3792, 128), dtype=uint8))]

In [5]:
import torch

In [2]:
!ls preprocessed_909

001.mid0.npz   187.mid11.npz  368.mid15.npz  550.mid3.npz   726.mid8.npz
001.mid10.npz  187.mid12.npz  368.mid1.npz   550.mid4.npz   726.mid9.npz
001.mid11.npz  187.mid13.npz  368.mid2.npz   550.mid5.npz   727.mid0.npz
001.mid12.npz  187.mid14.npz  368.mid3.npz   550.mid6.npz   727.mid10.npz
001.mid13.npz  187.mid15.npz  368.mid4.npz   550.mid7.npz   727.mid1.npz
001.mid14.npz  187.mid1.npz   368.mid5.npz   550.mid8.npz   727.mid2.npz
001.mid15.npz  187.mid2.npz   368.mid6.npz   550.mid9.npz   727.mid3.npz
001.mid1.npz   187.mid3.npz   368.mid7.npz   551.mid0.npz   727.mid4.npz
001.mid2.npz   187.mid4.npz   368.mid8.npz   551.mid10.npz  727.mid5.npz
001.mid3.npz   187.mid5.npz   368.mid9.npz   551.mid11.npz  727.mid6.npz
001.mid4.npz   187.mid6.npz   369.mid0.npz   551.mid1.npz   727.mid7.npz
001.mid5.npz   187.mid7.npz   369.mid1.npz   551.mid2.npz   727.mid8.npz
001.mid6.npz   187.mid8.npz   369.mid2.npz   551.mid3.npz   727.mid9.npz
001.mid7.npz   187.mid9.npz   369.mid3.npz   551.m

In [15]:
sample_path = os.path.join("preprocessed_909", "006.mid3.npz")
data = np.load(sample_path)
c_tensor = torch.tensor(data["c_tensor"], dtype=torch.long)
s_tensor = torch.tensor(data["s_tensor"], dtype=torch.bool)

In [16]:
c_tensor.shape

torch.Size([3, 256, 16, 2])

In [17]:
s_tensor.shape

torch.Size([3, 256])

In [18]:
s_tensor

tensor([[False, False, False, False, False, False, False,  True, False, False,
         False,  True, False, False, False,  True, False, False, False, False,
         False, False, False,  True, False, False, False, False, False, False,
         False,  True, False, False, False, False, False, False, False,  True,
         False, False, False,  True, False, False, False,  True, False, False,
         False, False, False, False, False,  True, False, False, False, False,
         False, False, False,  True, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True, False, False, False,  True, False, False,
         False,  True, False, False, False, False, False, False, False,  True,
         False, False, False,  True, False, False, F